In [22]:
import torch

torch.hub.list(github="ultralytics/yolov5", trust_repo='check')

Using cache found in /home/yungshun317/.cache/torch/hub/ultralytics_yolov5_master


['custom',
 'yolov5l',
 'yolov5l6',
 'yolov5m',
 'yolov5m6',
 'yolov5n',
 'yolov5n6',
 'yolov5s',
 'yolov5s6',
 'yolov5x',
 'yolov5x6']

In [23]:
# Check model information from `hubconf.py` in `ultralytics/yolov5`
# Official model will be downloaded to local as `yolov5s.pt`
yolo = torch.hub.load("ultralytics/yolov5", "yolov5s")
print(yolo)

Using cache found in /home/yungshun317/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-8 Python-3.12.3 torch-2.4.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 16072MiB)

100%|██████████████████████████████████████| 14.1M/14.1M [00:00<00:00, 18.7MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [24]:
import torch

# Image
image = 'https://ultralytics.com/images/zidane.jpg'

# Inference
results = yolo(image)
results.pandas().xyxy[0]

/home/yungshun317/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


,xmin,ymin,xmax,ymax,confidence,class,name
0,745.578735,48.471649,1142.671021,720.000000,0.869008,0,person
1,124.810516,197.328827,844.369141,716.658569,0.630537,0,person
2,441.253235,439.367798,498.379883,708.573730,0.616540,27,tie
3,594.089294,377.306091,635.425720,437.154663,0.274228,67,cell phone


In [27]:
print(results)

image 1/1: 720x1280 2 persons, 1 tie, 1 cell phone
Speed: 2283.7ms pre-process, 24.2ms inference, 50.8ms NMS per image at shape (1, 3, 384, 640)


In [13]:
from torchinfo import summary

# Peek inside the nested model
summary(yolo.model.model, input_size=(1, 3, 640, 640))

Layer (type:depth-idx)                        Output Shape              Param #
DetectionModel                                [1, 25200, 85]            --
├─Sequential: 1-1                             --                        --
│    └─Conv: 2-1                              [1, 32, 320, 320]         --
│    │    └─Conv2d: 3-1                       [1, 32, 320, 320]         (3,488)
│    │    └─SiLU: 3-2                         [1, 32, 320, 320]         --
│    └─Conv: 2-2                              [1, 64, 160, 160]         --
│    │    └─Conv2d: 3-3                       [1, 64, 160, 160]         (18,496)
│    │    └─SiLU: 3-4                         [1, 64, 160, 160]         --
│    └─C3: 2-3                                [1, 64, 160, 160]         --
│    │    └─Conv: 3-5                         [1, 32, 160, 160]         (2,080)
│    │    └─Sequential: 3-6                   [1, 32, 160, 160]         (10,304)
│    │    └─Conv: 3-7                         [1, 32, 160, 160]         (

In [ ]:
# Use `export.py` for converting `yolov5s.pt` to ONNX from Ultralytics's YOLOv5 repository

# !cd yolov5
# !git clone https://github.com/ultralytics/yolov5
# !cd yolov5
# !pip3 install ultralytics
# !python export.py --weights yolov5s.pt --workspace 2048 --data data/coco.yaml --include onnx --opset 12

# This will output a `yolov5s.onnx` file

In [ ]:
# /usr/src/tensorrt/bin/trtexec --onnx=yolov5s.onnx --saveEngine=/tensorfl_vision/Onnx-Inference-Yolov5/models_engine/yolov5s.engine

# YOLOv5

1: is due to that only 1 image has been processed
25200: is the number of anchors
85: is [x, y, w, h, objectness, 80 coco classes]

The 85 in the output tensor [1, 25556, 85] corresponds to the number of classes plus the bounding box coordinates and the objectness score for each prediction. In YOLOv5, the output tensor typically has the shape [batch_size, number_of_anchors, 4 + 1 + number_of_classes], where:

4 represents the bounding box coordinates (x, y, width, height),
1 represents the objectness score, and
number_of_classes is the number of classes the model is trained to detect.

To make sense of these outputs and convert them into a more usable form (bounding boxes, class IDs, and scores), you'll typically need to apply some post-processing steps. Here’s a brief overview:
- Apply a sigmoid function to the objectness scores and class predictions to convert logits to probabilities.
- Filter out predictions with objectness scores below a certain threshold to reduce the number of detections, as many will be low confidence.
- Apply Non-Max Suppression (NMS): Since your model may predict multiple overlapping boxes for a single object, NMS helps in selecting the most probable bounding box while discarding the rest.

1. `cv.dnn.readNetFromONNX()`
2. `cv.dnn.Net.getUnconnectedOutLayersNames()`: Returns names of layers with unconnected outputs.
3. `cv.dnn.Net.forward(outputName)`: Runs forward pass to compute output of layer with name `outputName`. By default runs forward pass for the whole network.
4. `cv.dnn.NMSBoxes(bboxes, scores, score_threshold, nms_threshold, eta=1.0, top_k=0)`: Performs non maximum suppression given boxes and corresponding scores. `eta` is a coefficient in adaptive threshold formula: $nms\_threshold_{i+1}=\eta⋅nms\_threshold_{i}$.

In [25]:
"""
Packages:
    cv2
    numpy
    time
    os
"""
import os
import cv2
import numpy as np
import time
import os

'''
    Target: Make inference With YOLOv5 Onnx model on set of images
    param[1]: path_to_images
    param[2]: path_to_model
    param[3]: image_height
    param[4]: image_width
    param[5]: confidence threshold
    param[6]: score threshold
    param[7]: nonmax suppression threshold
    param[8]: path_to_classes 
'''
class YOLOv5:
    # Initialize Declaration
    def __init__(self, path_to_images, path_to_model, image_width, image_height, conf_threshold, score_threshold, nms_threshold, path_to_classes):
        # Declare parameters
        self.path_to_images = path_to_images
        self.path_to_model = path_to_model
        self.image_width = image_width
        self.image_height = image_height
        self.conf_threshold = conf_threshold
        self.score_threshold = score_threshold
        self.nms_threshold = nms_threshold
        self.path_to_classes = path_to_classes

    # Python prebuilt `__call__` function
    '''
        target: Load images & ONNX model
        Return type : output list
    '''
    def __call__(self):
        for image in os.listdir(self.path_to_images):
            if image.endswith('.jpg') or image.endswith('.jpeg') or image.endswith('.png'):
                # Full absolute path to image
                image_full_path = os.path.join(self.path_to_images, image)
                # Read image
                image = cv2.imread(image_full_path)
                # Load ONNX model
                network = cv2.dnn.readNetFromONNX(self.path_to_model)
                # GPU configuration
                network.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
                network.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
                # Read classess
                classes = self.class_name()
                # Detection function
                self.detection(image, network, classes)

    # Read labels of COCO 
    '''
        target: read class labels
        Out: [classes list]
    '''
    def class_name(self):
        # List of classes
        classes = []
        file = open(self.path_to_classes, 'r')

        while True:
            name = file.readline().strip('\n')
            classes.append(name)
            if not name:
                break
        return classes
    
    '''
    target: To make inference and show image
    name : detection
    param[1]: image
    param[2]: network
    param[3]: classes
    '''
    def detection(self, image, net, classes):
        # Blob to apply to input image
        # Out[]: Return 4-dimensional matrix with NCHW dimensions order 
        blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (640, 640), swapRB=True, mean=(0, 0), crop=False)

        # Set to input model
        # Sets the new input value for the network
        net.setInput(blob)

        # Start time
        t_0 = time.time()

        # Unconnected layer by index
        output_layers = net.getUnconnectedOutLayersNames()
        # print(output_layers)
        # ('output0',)

        # Out[]: Blob for first output of specified layer
        # Detection results
        outputs = net.forward(output_layers)
        # print(type(outputs))
        # <class 'tuple'>
        # print(outputs[0].shape)
        # (1, 25200, 85)

        # If not using `getUnconnectedOutLayersNames()`
        # outputs = net.forward()
        # print(type(outputs))
        # <class 'numpy.ndarray'>
        # print(outputs.shape)
        # (1, 25200, 85)
        
        # End time
        t_1 = time.time()
        print('OpenCV DNN YOLOv5 Inference Time:', t_1 - t_0)

        # Number of detections
        num_detections = outputs[0].shape[1]
        # print(num_detections)
        # 25200

        height, width = image.shape[:2]
        # Scale
        x_scale = width / self.image_width
        y_scale = height / self.image_height

        # Confidence
        confidence_threshold = self.conf_threshold
        # Score
        score_threshold = self.score_threshold
        # Non max suppresion 
        nms_threshold = self.nms_threshold

        # List of class ids
        class_ids = []
        confidences = []
        bboxes = []

        # Loop through detections
        for i in range(num_detections):
            # Detect 
            detect = outputs[0][0][i]
            # Confidence
            confidence = detect[4]

            if confidence >= confidence_threshold:
                # Scores of each class
                class_score = detect[5:]
                class_idx = np.argmax(class_score)

                if (class_score[class_idx] > score_threshold):
                    confidences.append(confidence)
                    class_ids.append(class_idx)

                    cx, cy, w, h = detect[0], detect[1], detect[2], detect[3]

                    # Calculate bounding box coordinates
                    left = int((cx - w / 2) * x_scale)
                    top = int((cy - h / 2 ) * y_scale)
                    width = int(w * x_scale)
                    height = int(h * y_scale)

                    box = np.array([left, top, width, height])
                    bboxes.append(box)

        # Non max suppression 
        indices = cv2.dnn.NMSBoxes(bboxes, confidences, confidence_threshold, nms_threshold)

        for i in indices:
            box = bboxes[i]
            left = box[0]
            top = box[1]
            width = box[2]
            height = box[3]

            # Label
            label = "{}:{:.2f}".format(classes[class_ids[i]], confidences[i])
                            
            # Rectangle
            cv2.rectangle(image, (left, top), (left + width, top + height), (0, 255, 0), 3)
            # Task for learning put text 
            cv2.putText(image, label, (left, top + 20), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)

        cv2.namedWindow('detection.jpg', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('detection.jpg', 900, 800)
        cv2.imshow('detection.jpg', image)

        cv2.waitKey(3000)
        cv2.destroyAllWindows()
        
def main():
    path_to_images = 'yolov5/images'
    path_to_model = 'yolov5/yolov5s.onnx'
    path_to_classes = 'yolov5/coco-classes.txt'
    
    instance = YOLOv5(path_to_images=path_to_images, path_to_model=path_to_model, image_width=640, image_height=640, conf_threshold=0.34, score_threshold=0.38, nms_threshold=0.3, path_to_classes=path_to_classes)
    instance()

if __name__== "__main__":
    main()

OpenCV DNN YOLOv5 Inference Time: 0.07496237754821777
OpenCV DNN YOLOv5 Inference Time: 0.0379633903503418
OpenCV DNN YOLOv5 Inference Time: 0.03557896614074707
OpenCV DNN YOLOv5 Inference Time: 0.03752326965332031
OpenCV DNN YOLOv5 Inference Time: 0.03410220146179199
OpenCV DNN YOLOv5 Inference Time: 0.034317731857299805
OpenCV DNN YOLOv5 Inference Time: 0.038039207458496094
OpenCV DNN YOLOv5 Inference Time: 0.034429073333740234


In [28]:
import tensorrt as trt
import pycuda.autoinit
import pycuda.driver as cuda
import cv2
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import yaml
import time

class TRTInference:    
    # Specify engine file path and input and output shape
    def __init__(self, path_to_engine_file, input_shape, output_shape, path_to_class_file, conf_threshold, score_threshold, nms_threshold):
        self.logger = trt.Logger(trt.Logger.WARNING)
        
        self.path_to_engine_file = path_to_engine_file
        # Load engine
        self.engine = self.load_engine(self.engine_file_path)
        
        # Create context
        self.context = self.engine.create_execution_context()

        self.conf_threshold = conf_threshold
        self.score_threshold = score_threshold
        self.nms_threshold = nms_threshold

        # Input shape
        self.input_shape = input_shape
            
        self.path_to_class_file = path_to_class_file
        self.count = 0
            
        # Output shape
        self.output_shape = output_shape

        # Read classes
        # with open(path_to_class_file, 'r') as read_class:
        #     self.class_labels = [line.strip() for line in read_class.readlines()]
        with open(path_to_class_file, 'r') as read_class:
            data = yaml.safe_load(read_class)
            self.class_labels = [name for name in data['names'].values()]

    def load_engine(self, path_to_engine_file):
        with open(path_to_engine_file, 'rb') as f:
            runtime = trt.Runtime(self.logger)
            deserialized_engine = runtime.deserialize_cuda_engine(f.read())
            return deserialized_engine
            
    def preprocess_image(self, image_path):
        image_list = []
        image_path = []
        count = 0
       
        for original_image in os.listdir(image_path):
            if original_image.endswith('.jpg') or original_image.endswith('.png') or original_image.endswith('jpeg'):
                image_full_path = os.path.join(image_path, original_image)
                self.image = cv2.imread(image_full_path)

                self.original_h, self.original_w = self.image.shape[:2]
           
                # `image_size = [640, 640]`
                self.resized_image = cv2.resize(self.image, (self.input_shape[2], self.input_shape[3]), interpolation=cv2.INTER_AREA)
                # self.resized_image = self.resize_with_aspect_ratio(self.image, target_size=(self.input_shape[2], self.input_shape[3]))
                image_array = np.array(self.resized_image).astype(np.float32) / 255.0

                image_array = image_array.transpose((2, 0, 1))

                image_array = np.expand_dims(image_array, axis=0)

                self.resized_image_h, self.resized_image_w = self.resized_image.shape[:2]               

                count += 1
                
                image_list.append(image_array)
                image_path.append(image_full_path)

                if count >= 12:
                    continue
                    
        return  image_list, image_path
        
    
    def inference_detection(self, image_path):     
        input_list, full_img_paths = self.preprocess_image(image_path)

        results = []
        stream = cuda.Stream()

        self.total_time = 0
        self.num_frames = len(input_list)

        for inputs, full_img_path in zip(input_list, full_img_paths):

            # Start time
            self.start = time.time()

            inputs = np.ascontiguousarray(inputs)
            outputs = np.empty(self.output_shape, dtype=np.float32)
            d_inputs = cuda.mem_alloc(1 * inputs.nbytes)
            d_outpus = cuda.mem_alloc(1 * outputs.nbytes)
            bindings = [d_inputs, d_outpus]
      
            '''cuda.memcpy_htod_async(d_inputs, inputs, stream)

            self.context.execute_async(bindings=bindings, stream_handle=stream.handle)

            cuda.memcpy_dtoh_async(outputs, d_outpus, stream)

            stream.synchronize()'''

            cuda.memcpy_htod(d_inputs, inputs)
            self.context.execute_v2(bindings)

            # Copy output back to host
            cuda.memcpy_dtoh(outputs,d_outpus)

            # cuda.memcpy_htod_async(d_outpus, outputs, stream)
            # result = self.postprocess_img(outputs)      
         
            d_inputs.free()
            d_outpus.free()

            # End time
            self.end = time.time()
            self.total_time  += (self.end - self.start)
            self.fps = self.num_frames / self.total_time
            self.postprocess_recognized_image(full_img_path, outputs)    

        return outputs
        
    # Save images with detected results
    def postprocess_recognized_image(self, image_path, outputs):
        # image = Image.open(image_path)
        image = cv2.imread(image_path)
        # image = image.cop()
       
        num_detections = outputs[0].shape[0]
        # print(outputs[0][0][0])
        
        width, height =  image.shape[:2]

        # Re-scaling
        x_scale = width / self.resized_image_w
        y_scale = height / self.resized_image_h

        # width, height =  self.image.shape[:2]
        conf_threshold = self.conf_threshold
        score_threshold = self.score_threshold
        nms_threshold = self.nms_threshold

        class_ids = []
        confidences = []
        bboxes = []
        # print(outputs)

        for i in range(num_detections):
            detect = outputs[0][i]
            # print(detect)
            
            conf = detect[4]
          
            if conf >= conf_threshold:
                class_score = detect[5:]  
                class_idx = np.argmax(class_score)

                if (class_score[class_idx] > score_threshold):
                    # Confidence
                    confidences.append(getConf)
                    class_ids.append(class_idx)

                    # Get center and w, h coordinates
                    cx, cy, w, h = detect[0], detect[1], detect[2], detect[3]
                    # print("Center X",cx, "Center Y ", cy, " Width", w, "Height: ", h)
                    # print('*********************************************************')
                    # print('\n')
                    
                    # Left
                    left = int((cx - w/2) * x_scale)

                    # Top
                    top = int((cy - h/2) * y_scale)

                    # Width
                    width = int(w * x_scale)

                    # Height
                    height = int(h * y_scale) 
                    
                    # Box
                    box = np.array([left, top, width, height])

                    # Bounding boxes
                    bboxes.append(box)
                    
        # print("output of box")
        # print(bboxes)
        # get max suppresion
        indices = cv2.dnn.NMSBoxes(bboxes, confidences, conf_threshold, nms_threshold)

        for i in indices:
            box = bboxes[i]
            left = box[0] 
            top = box[1] 
            width = box[2] 
            height = box[3]

            print("Box Left ", left , "Box Top ", top, "Box Width ", width, "Box height: ", height)
            print('*********************************************************')
            print('\n')
        
            print(self.class_labels[class_ids[i]])
            print()
            label = "{}:{:.2f}".format(self.class_labels[class_ids[i]], confidences[i])

            # label2 = "FPS: {}".format(self.fps)

            cv2.rectangle(image, (left, top),(left + width, top + height), (0, 255, 0), 3)
            cv2.putText(image, label, (left, top - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
           
        cv2.namedWindow('result.jpg', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('result.jpg', 900, 800)
        cv2.imshow('result.jpg', image)
        cv2.waitKey(2000)
        cv2.destroyAllWindows()

def main():
    path_to_engine_file ='/home/yungshun317/workspace/py/onnx-tensorrt-inference/yolov5'

    # Load the TensorRT engine
    input_shape = (1, 3, 640, 640)
    # output_shape = (1, 25500, 7)
    output_shape = (1, 25200, 85)

    # image_path = '/deeplearning/resnet/rose.jpeg'
    image_path = '/home/yungshun317/workspace/py/onnx-tensorrt-inference/yolov5/images'
    path_to_class_file = "/home/yungshun317/workspace/py/onnx-tensorrt-inference/yolov5/yolov5/data/coco.yaml"
    
    inference = TRTInference(path_to_engine_file=path_to_engine_file, input_shape=input_shape, output_shape=output_shape, path_to_class_file=path_to_class_file, conf_threshold=0.4, score_threshold=0.45, nms_threshold=0.35)
    inference.inference_detection(image_path)

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'tensorrt'

In [ ]:
'''
    Project : yolov5 Inference on video frames

'''
# import preprocessing libraries

import tensorrt as  trt
import pycuda.autoinit
import cv2
import pycuda.driver as cuda

import numpy as np

import os
from PIL import Image
import matplotlib.pyplot as plt

import yaml
import time


'''
    Class Name: yolov5TensorRT
    target: INIT Class params
    param[1]: engine_file_path
    param[2]: input_shape
    param[3]: output_shape
    param[4]: classes_label_file
    param[5]: conf_threshold
    param[6] : score_threshold
    param[7] : nms_threshold

'''

class yolov5TensorRT:
   
    def __init__ (self, engine_file_path, input_shape, output_shape, classes_label_file, conf_threshold, score_threshold, nms_threshold):
        
        # Warning while engien loading
        self.logger = trt.Logger(trt.Logger.WARNING) # ?
        
        # INIt Params
        self.engine_file_path = engine_file_path
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.classes_label_file = classes_label_file
        self.conf_threshold = conf_threshold
        self.score_threhold = score_threshold
        self.nms_threshold = nms_threshold
        
        # load engine
        self.engine = self.load_engine(self.engine_file_path)
        self.context = self.engine.create_execution_context()
        
    
        # read class labels of yaml
        # return all classes 
        with open(classes_label_file, 'r') as class_read:
            data = yaml.safe_load(class_read)
            self.class_labels = [name for name in data['names'].values()]
    
    ''' 
        loading engine file and deserialize for an inference
        param[1]: engine_file_path
        param[out]: deserialized_engine
    '''
    def load_engine(self,engine_file_path):
        with open(engine_file_path, 'rb') as f:
            
            runtime = trt.Runtime(self.logger)
            engine_deserialized = runtime.deserialize_cuda_engine(f.read())
            
        return engine_deserialized
    
    '''
        target:preprocessing video frames
        param[1]: video_path
        param[out]: video frames
    '''
    
    def preprocess_video(self,video_path):
        
        video = cv2.VideoCapture(video_path)
        
        # while opens, do preprocessing
        while video.isOpened():
            
            ret, frame = video.read()
            
            frame = cv2.resize(frame, (1000, 800))
            
            
            if not ret:
                
                break
            
            self.org_frame_h, self.org_frame_w = frame.shape[:2]
            
            
            '''
                NORMALIZATION FOR INFERENCE
            '''
            img_resized = cv2.resize(frame, (self.input_shape[2], self.input_shape[3]), interpolation=cv2.INTER_AREA)
            
            self.resized_frame_h, self.resized_frame_w = img_resized.shape[:2]
            
            # convert to n umpy and divide it by float32 bit and 255.0
            
            img_np = np.array(img_resized).astype(np.float32) / 255.0
            
            img_np = np.transpose(img_np, (2, 0, 1))
            
            yield img_np, frame
            
        video.release()
        
    '''
        param[1]: video path
        param[out1]:frame
        param[out2]:outputs 
    '''
    def inference_detection(self,video_path):
        
        self.total_time = 0
        
        self.num_frames = 0
        
        
        for inputs, frame in self.preprocess_video(video_path):
            
            self.num_frames += 1
            
            self.start = time.time()
            
            inputs = np.ascontiguousarray(inputs)
            
            outputs = np.empty(self.output_shape, dtype=np.float32)
            
            
            d_inputs = cuda.mem_alloc(1 * inputs.nbytes)
            
            d_outputs = cuda.mem_alloc(1 * outputs.nbytes)

            bindings = [d_inputs, d_outputs]
            
            cuda.memcpy_htod(d_inputs, inputs)
            
            self.context.execute_v2(bindings)
            
            cuda.memcpy_dtoh(outputs, d_outputs)
            
            d_inputs.free()
            d_outputs.free()
            
            # end time 
            
            self.end = time.time()
            
            self.total_time += (self.end - self.start)
            
            self.FPS = self.num_frames / self.total_time
            
            # post processing gpu results
            
            self.postprocessing_recognized_frames(frame, outputs)


        
        return outputs
        
        
    '''
    target: postprocessing
    param[1]: frame
    param[2]: yolov5 output gpu results
    '''    
    def postprocessing_recognized_frames(self, frame, yolov5_output):
        
        
        detections = yolov5_output[0].shape[0]
        
        width, height = frame.shape[:2]
        
        x_scale = self.org_frame_w / self.resized_frame_w
        y_scale = self.org_frame_h / self.resized_frame_h
        
        conf_threshold = self.conf_threshold
        
        score_threshold = self.score_threhold
        
        nms_threshold= self.nms_threshold
        
        class_ids = []
        
        confidences = []
        
        bboxes = []
        
        
        for i in range(detections):
            
            detect = yolov5_output[0][i]
            
            getConf = detect[4]
            
            if getConf >= conf_threshold:
                
                class_score = detect[5:]
                
                class_idx = np.argmax(class_score)
                
                if (class_score[class_idx] > score_threshold):
                    confidences.append(getConf)
                    
                    class_ids.append(class_idx)
                    
                    # yolov5 output formats 
                    cx, cy, w, h = detect[0], detect[1], detect[2], detect[3]
                    
                    left = int((cx - w / 2) * x_scale)
                    top = int((cy - h / 2) * y_scale)
                    
                    width =  int(w * x_scale)
                    height = int(h * y_scale)
                    
                    box = np.array([left, top, width, height])
                    
                    bboxes.append(box)
                    
                    
        
        indices_nonmax = cv2.dnn.NMSBoxes(bboxes, confidences, conf_threshold, nms_threshold)
        
        for i in indices_nonmax:
            box = bboxes[i]
            left = box[0]
            top = box[1]
            width = box[2]  
            height = box[3]
            
            
            label = "{}:{:.2f}, FPS: {:.2f}".format(self.class_labels[class_ids[i]], confidences[i], self.FPS)
            
            cv2.rectangle(frame, (left, top), (left + width, top+height), (0, 255, 0), 3)
            cv2.putText(frame, label, (left, top-20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
            
            
            cv2.imshow('Detection.jpg', frame)
        
            if cv2.waitKey(1) & 0xFF == ord('q'):
                cv2.destroyAllWindows() 
            
            
def main():
    engine_file_path =  '/tensorfl_vision/TensorRT_Inference/Models/yolov5s.engine'
    input_shape = (1, 3, 640, 640)
    output_shape = (1, 25200, 85)
    video_path = '/tensorfl_vision/Yolov5_Video_Inference/main_demo.mp4'
    
    path_to_classes = '/tensorfl_vision/TensorRT_Inference/coco.yaml'
    
    inference = yolov5TensorRT(engine_file_path, input_shape, output_shape,  path_to_classes, 0.4, 0.45, 0.35)
    
    inference.inference_detection(video_path)
    
    
if __name__=="__main__":
    main()
    
    